# Yelp API Webscraping

add echo's comment

## aerika section

In [1]:
hi

NameError: name 'hi' is not defined

In [2]:
my

NameError: name 'my' is not defined

In [3]:
name

NameError: name 'name' is not defined

## echo section

added comment

In [ ]:
added cell

## jerry section

In [ ]:
# ds;alkfjds;lfkas;dghas'dlfjas;dlkfjas'dfj

In [1]:
# this is another change a;sdfja;dfkasd;fajsd;fasdkfjas;fwef

In [2]:
# yet another change ;asdlfkjas;dknva;djaweoifsd;laknfas;dfknasd

In [3]:
x = np.exp(52)
x

3.831008000716577e+22